In [1]:
# Imports
import pandas as pd
import re
from datetime import datetime, timedelta
from fake_useragent import UserAgent
import time

import requests
from bs4 import BeautifulSoup as soup
import lxml

import json
#from dateutil import rrule

## needs a py file to import properly in this structure
#from utils.collection_utils import datetime_parse

from itertools import combinations, permutations, chain


In [2]:
DATA_PATH = '../data/'
IN_DATA_PATH = '../data/input_data/'

### !!! Datetime function needs work

If the month dict makes a replacement, need to tag that as a month and limit my permutations as required

In [3]:
import re
from itertools import combinations, permutations, chain
from datetime import datetime, timedelta

def datetime_parse(x):
      '''
      Parse datetime out of a string
      More functionality should be added as issues encountered 
      Has some serious issues as it stands... not a hard fix, but time consuming 
      '''
      # Remove all non-alpha-numeric
      out = re.sub(r'[^0-9a-zA-Z:]+', ' ', x)

      ## Remove any starting tag with :
      # out.split('Updated: ', expand=True)
      
      # Remove time 
      out.replace(r'\b(([0-9]|0[0-9]|1[0-9]|2[0-3]):[0-5][0-9](:[0-5][0-9])?\s?([AaPp][Mm])?)', ' ')
      
      # Not needed
      month_dict = {
      'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May':'5', 'June':'6',
      'July':'7', 'August':'8', 'September':'9', 'October':'10', 'November':'11', 
      'December':'12', 'Jan':'1', 'Feb':'2', 'Mar':'3', 'Apr':'4', 'May':'5',
      'Jun':'6', 'Jul':'7', 'Aug':'8', 'Sep':'9', 'Oct':'10', 'Nov':'11', 'Dec':'12'
      }

      # Not needed, replaces months with number value, but strtime does that
      #out = [out.replace(key, value) for key, value in month_dict.items() if key in out][0]
      
      # Removing digits longer than 4 long
      out = re.sub(r'[0-9]\d{4,}', ' ', out)
      #out = re.sub(r'[0-9]+:[0-9]+', ' ', out) #  not sure what this was for
      
      # Removing all non-digits and words not in months
      out = out.split(' ')
      out = [word for word in out if word.isdigit() or word in list(month_dict.keys())]
      out = ' '.join(out[:3])
      
      # Strip loose whitespace
      out = out.strip()

      # Lists for parsing
      month = ['%b', '%m', '%B']
      day = ['%d']
      year = ['%Y', '%y']

      varieties = list(permutations(chain(year, day, month), 3))
      for v in varieties:
            v = ' '.join(v)
            try:
                  date = [datetime.strptime(str(out), v)]# if d != 0 else d for d in out] #old tag used elsewhere, here for ref.
                  print(date)
                  if date is not None:
                        print('Successfully parsed with format: ' + v)
                        return date[0]
                        break
            except:
                  #print('Failed: ' + v)
                  pass

In [ ]:
str1 = '2020 01 23'
date = datetime_parse(str1)
date

In [ ]:
date.month

In [ ]:
# # To load as dataframe: no value at present 

# load_file = IN_DATA_PATH + 'collection_urls_df.jsonl'

# df = pd.read_json(load_file, convert_dates=True, lines=True, orient='records')
# df.head(7)

In [ ]:
load_file = IN_DATA_PATH + 'collection_urls_dict.json'

with open(load_file) as handle:
    sources = json.loads(handle.read())

In [ ]:
sources

In [ ]:
# Get today's date
# To do: create a "last scraped json"
today = datetime.now()

today

# Import AI

In [ ]:
# Lists for saving collected data

title_list = []
url_list = []
dates = []


In [ ]:
base_url = sources['Import_AI']

In [ ]:
# Get list of all Import AI issues with date, title, and link

response = requests.post(base_url) #headers=header)

html = soup(response.text, 'lxml')

objects = html.find_all('li', class_="campaign")
for obj in objects:
      print(obj.text)
      print(obj.a['href'])
      title_list.append(obj.text)
      url_list.append(obj.a['href'])

print('------------')

In [ ]:
title_list

In [ ]:
# Parse date from list

for item in title_list:
      date = item.split(' -')[0]
      print(date)
      date_parsed = datetime.strptime(date, '%m/%d/%Y')
      print(date_parsed)
      #date_sequence = datetime_parse(item)
      #dates.append(date_sequence[0])
      dates.append(date_parsed)

In [ ]:
# Save collection to dataframe

df_collected = pd.DataFrame(list(zip(title_list, url_list, dates)), 
            columns=['title', 'url', 'date'])

df_collected.head(20)

In [ ]:
df_collected.columns

In [ ]:
# Enter keywords in json referenced here

# For testing only
#search_terms = ['DeepMind', 'Google']

load_file = IN_DATA_PATH + 'collection_searchterms.json'

with open(load_file) as handle:
    search_terms = json.loads(handle.read())

In [ ]:
search_terms['search_term']

In [ ]:
# List to save to
relevant_text = []

#preping list fo dictionary conversion
relevant_text.append(['title', 'url', 'date', 'text'])


In [ ]:
# Gets text for results within timedelta window
#     This is probably best changed to a "last scraped" date from a json

for index, row in df_collected.iterrows():
      if row.date >= today - timedelta(days=30):
            print('Fetching: ' + row.url)
            response = requests.post(row.url) #headers=header)

            html = soup(response.text, 'lxml')

            objects = html.find_all('p')#, class_="campaign")
            issue_text = []
            for obj in objects:
                  issue_text.append(obj.text)
                  #print(obj.text)
            issue_text = ' '.join(issue_text)
            #print(issue_text)
            #print('--------------------------------')
            
            for word in search_terms['search_term']:
                  if word in issue_text:
                        save = list(row)
                        save.append(issue_text)
                        relevant_text.append(save)                  
                        print('Search term found: ' +  word)
                        break
                  else: 
                        pass
            time.sleep(5)
      else:
            pass


In [ ]:
len(relevant_text)

In [ ]:
## To save to json - not recommend as datetimes not serializable, would have to convert dt back to string
# Note - ugly dictionary, e.g. not jsonl or indented, but kept simple unless requested

# data_dict = {k: v for k, *v in zip(*relevant_text)}

# with open('data.json', 'w') as f:
#     json.dump(data_dict, f)

In [ ]:
new_df = pd.DataFrame(relevant_text[1:],columns=relevant_text[0])

In [ ]:
# TO DO: add if file clause
old_df = pd.read_csv(DATA_PATH + 'open_ai_data.csv')

In [ ]:
combined_df = pd.concat([new_df, old_df])

In [ ]:
# If timing for collection and timedelta etc are good, should not be needed, but good to be sure
# If you instead have date issues, you should do this in the collect loop to eliminate unneeeded collectio
# e.g. if url in list(old_df.url): pass

combined_df.drop_duplicates(subset='url', inplace=True)

In [ ]:
combined_df

In [ ]:
combined_df.to_csv(DATA_PATH + 'open_ai_data.csv', index=False)

# Synced

In [ ]:
base_url = sources['Synced']

In [ ]:
# just checking
today

In [ ]:
# Get date part for url
year = datetime.now().year
month = datetime.now().month

In [ ]:
current_url = base_url + str(year) + '/' +  str(month) + '/'
current_url


In [ ]:
# lists for saving
title_list = []
url_list = []
summary_list = []
date_list = []

In [ ]:
# Setting up a fake useragent
# Can do for all collections, but for now limited to where is seems necessary only
# Ignore the error 

ua = UserAgent(verify_ssl=False, cache=False)

user_agent = ua.random
header = {'User-Agent': user_agent}
print(header)

In [ ]:
response = requests.post(url, headers=header)

html = soup(response.text, 'lxml')

obj = html.find('div',id= "primary") 

In [ ]:

dates = obj.find_all(class_='entry-date')
for d in dates:
      d_parsed = datetime_parse(d.text)
      date_list.append(d_parsed[0])
      #date_list.append(d.text)

titles = obj.find_all(class_='entry-title')
for t in titles:
      title_list.append(t.text)
      url_list.append(t.a['href'])

summaries = obj.find_all(class_='entry-summary')
for s in summaries:
      summary_list.append(s.text)



In [ ]:
df_collected2 = pd.DataFrame(list(zip(title_list, url_list, date_list, summary_list)), 
            columns=['title', 'url', 'date', 'summary'])

df_collected2.tail(2)

In [ ]:
#keyword = 'AlexNet'
search_terms['search_term']

In [ ]:
relevant_text = []

relevant_text.append(['title', 'url', 'date', 'summary', 'text'])


In [ ]:
for index, row in df_collected2.iterrows():
      if row.date >= today - timedelta(days=14):
            print('Fetching: ' + row.url)
            response = requests.post(row.url) #headers=header)

            html = soup(response.text, 'lxml')

            objects = html.find_all('div', class_="entry-content")
            article_text = []
            for obj in objects:
                  paragraph_text = obj.find_all('p')
                  for p in paragraph_text:
                        article_text.append(p.text)
                  #print(obj.text)
            article_text = ' '.join(article_text)

            for word in search_terms['search_term']:
                  if word in article_text:
                        save = list(row)
                        save.append(article_text)
                        relevant_text.append(save)                  
                        print('Search term found: ' +  word)
                        break
            else: 
                  pass

            time.sleep(5)
      else:
            pass


In [ ]:
new_df = pd.DataFrame(relevant_text[1:],columns=relevant_text[0])

In [ ]:
new_df

In [ ]:
# TO DO: add if file clause
old_df = pd.read_csv(DATA_PATH + 'synced_data.csv')

In [ ]:
combined_df = pd.concat([new_df, old_df])

In [ ]:
# If timing for collection and timedelta etc are good, should not be needed, but good to be sure
# If you instead have date issues, you should do this in the collect loop to eliminate unneeeded collectio
# e.g. if url in list(old_df.url): pass

combined_df.drop_duplicates(subset='url', inplace=True)

In [ ]:
combined_df

In [ ]:
combined_df.to_csv(DATA_PATH + 'synced_data.csv', index=False)

# MIT

In [ ]:
base_url = sources['MIT_Technology_Review']

In [ ]:
topic_designator = 'topic/'
topic_tags = ['artificial-intelligence', 'humans-and-technology', 'computing'] 

In [ ]:
final_url = base_url + topic_designator + topic_tags[0]
final_url

In [ ]:
for tag in topic_tags:
      final_url = base_url + topic_designator + tag
      # then search

In [ ]:
response = requests.get(final_url)

# print headers of response
print(response.headers)

In [ ]:
API_ROOT_URL = "https://wp.technologyreview.com/wp-json/irving/v1/data/term_archive?category_name=artificial-intelligence&page=1"
'''
API_ROOT_URL
slug
?
requestType
=
query
&page=1'''

def searchApi():
    endpoint = API_ROOT_URL
    data = {
        "slug": "term_archive",
        "requestType": "category_name",
        "query": "artificial-intelligence"
    }
    try:
        response = requests.post(endpoint)#, data=data)
        if(response.status_code == 200):
            #print(response.json())
            for msg in response:
                print(msg)
    except Exception:
        print(traceback.format_exc())

In [ ]:
searchApi()


In [ ]:
API_Link = "https://wp.technologyreview.com/wp-json/irving/v1/data/term_archive?category_name="
#artificial-intelligence"
#&page=1"

In [ ]:
# Making a get test request
try:
      response = requests.get(API_Link)

      # print headers of response
      if(response.status_code == 200):
            print(response.headers)
            
except Exception:
      print(traceback.format_exc())


In [ ]:
def get_response(API_Link, pages=1):
    with requests.get(API_Link + '&page=' + str(pages)) as response:
        page_soup = soup(response.content, 'lxml')
        return page_soup

In [ ]:
responses = []

for tag in topic_tags:
      response = get_response(API_Link + tag, pages=str(1))

      #parse
      j_response = json.loads(response.text)
      responses = responses + j_response

In [ ]:
responses[1].keys()

In [ ]:
responses[1]['config'].keys()

In [ ]:
#initial empty df
collected_df = pd.DataFrame(columns=['title', 'url', 'date', 'summary'])
collected_df.loc[collected_df.index, :] = ['x', 'url', 'date', 'summary']

collected_df

In [ ]:
needed_keys = ['title', 'permalink', 'postDate', 'excerpt']

entries = []
entries.append(['title', 'url', 'date', 'summary'])

for item in responses:
      entry = [item['config'][key] for key in needed_keys]
      entries.append(entry)
      #collected_df.loc[collected_df.index.max() + 1, :] = entry


In [ ]:
new_df = pd.DataFrame(entries[1:],columns=entries[0])

In [ ]:
def date_from_url(x):

      pat = r"(20[0-2][0-9]([-_/]?)[0-9]{2}(?:\2[0-9]{2})?)"
      dates = re.compile(pat)

      res = dates.search(x)

      res = datetime_parse(res[0])
  
      return res[0]


In [ ]:
len(new_df)

In [ ]:
new_df['date'] = new_df['url'].apply(lambda x: date_from_url(x))


In [ ]:
relevant_text = []

relevant_text.append(['title', 'url', 'date', 'summary', 'text'])


In [ ]:
for index, row in new_df.iterrows():
      if row.date >= today - timedelta(days=7):
            print('Fetching: ' + row.url)
            response = requests.post(row.url, headers=header)

            html = soup(response.text, 'lxml')

            objects = html.find_all('div', id="content--body")
            article_text = []
            for obj in objects:
                  paragraph_text = obj.find_all('p')
                  for p in paragraph_text:
                        article_text.append(p.text)
                  #print(obj.text)
            article_text = ' '.join(article_text)

            for word in search_terms['search_term']:
                  if word in article_text:
                        save = list(row)
                        save.append(article_text)
                        relevant_text.append(save)                  
                        print('Search term found: ' +  word)
                        break
            else: 
                  pass

            time.sleep(5)
      else:
            pass


In [ ]:
new_df2 = pd.DataFrame(relevant_text[1:],columns=relevant_text[0])

In [ ]:
# TO DO: add if file clause
old_df = pd.read_csv(DATA_PATH + 'synced_data.csv')

In [ ]:
combined_df = pd.concat([new_df, old_df])

In [ ]:
# If timing for collection and timedelta etc are good, should not be needed, but good to be sure
# If you instead have date issues, you should do this in the collect loop to eliminate unneeeded collectio
# e.g. if url in list(old_df.url): pass

combined_df.drop_duplicates(subset='url', inplace=True)

In [ ]:
combined_df.to_csv(DATA_PATH + 'mit_data.csv', index=False)

# IEEE

### Currently not working well - only 4 returns due to dynamic scrolling
I can solve with Selenium, but trying to avoid using it

Currently working on a requests version, but convoluated urls and html

In [ ]:
## Can get all the rest (set range based on date) but missed first page returned

counter = 0
for i in range(1, 5): #while
      url = "https://spectrum.ieee.org/res/load_more_posts/data.js?site_id=20265424&node_id=/root/blocks/block[search]/abtests/abtest[1]/row/column[1]/choose/when[getparams.order]/choose/otherwise/element_wrapper-&resource_id=search_deepfake&path_params={}&formats=html&q=deepfake&topic=&order=newest&rm_lazy_load=16&pn=%s&pn_strategy=" % (counter)
      #url = "https://spectrum.ieee.org/res/load_more_posts/data.js?site_id=20265424&node_id=/root/blocks/block[search]/abtests/abtest[1]/row/column[1]/choose/otherwise/choose/otherwise/element_wrapper-&resource_id=search_deepfake&path_params={}&formats=html&q=deepfake&order=newest&rm_lazy_load=1&pn=%s&pn_strategy=" % (counter)
      print(url)
      counter +=1
      # ua = UserAgent(verify_ssl=False, cache=False)

      # user_agent = ua.random
      # header = {'User-Agent': user_agent}

      response = requests.post(url)#, headers=header)

      html = soup(response.text, 'lxml')

      objects = html.find('div')#, class_='posts-wrapper')#elid=True)#class_= 'clearfix')

      for obj in objects:
            items = obj.find_all('span')
            for it in items:
                  print(it.text)
            items = obj.find_all('h2')#, class_ = 'widget__head')#, id='col-right')
            #print(len(items))
            for i in items:
                  print(i.text)
                  print(i.a['href'])

### Not great

In [ ]:
base_url ='https://spectrum.ieee.org/'

In [ ]:
query_url = 'https://spectrum.ieee.org/search/?q='

In [ ]:
topic_url = 'https://spectrum.ieee.org/topic/artificial-intelligence/'

In [ ]:
# query connector
C = "&"

In [ ]:
sort_criteria = "order=newest"

In [ ]:
topic_criteria = "topic=artificial-intelligence"

In [ ]:
# initialized above
today

In [ ]:
today.date()

In [ ]:
back_to = today.date() - timedelta(days=30)
back_to

In [ ]:
search_terms['search_term']

In [ ]:
for term in search_terms['search_term']:
      final_url = query_url + term + C + sort_criteria
      print(final_url)

In [ ]:
print(final_url)
ua = UserAgent(verify_ssl=False, cache=False)

user_agent = ua.random
header = {'User-Agent': user_agent}

response = requests.post(final_url, headers=header)

html = soup(response.text, 'lxml')


In [ ]:
title_list = []
url_list = []
date_list = []

In [ ]:
# Works some at least but only fetches top 4 due to page scroll

objects = html.find_all('div',  class_='section_column')#elid=True)#class_= 'clearfix')

for obj in objects:
      items = obj.find_all('div', id='col-right')
      print(len(items))
      for i in items:

            print(i.h2.text)
            title_list.append(i.h2.text)

            print(i.h2.a['href'])
            url_list.append(i.h2.a['href'])

            date = i.div.span
            print(date.text)
            try: 
                  #not working well
                  d_parsed = datetime_parse(date.text)
                  date_list.append(d_parsed[0])
            except:
                  date_list.append('None')


In [ ]:
df_collected3 = pd.DataFrame(list(zip(title_list, url_list, date_list)), 
            columns=['title', 'url', 'date',])

df_collected3.head(4)

In [ ]:
# List to save to
relevant_text = []

#preping list fo dictionary conversion
relevant_text.append(['title', 'url', 'date', 'text'])

In [ ]:
today

In [ ]:
for index, row in df_collected3.iterrows():
      if row.date >= today - timedelta(days=30):
            print('Fetching: ' + row.url)
            response = requests.post(row.url) #headers=header)

            html = soup(response.text, 'lxml')

            objects = html.find_all('div', id="col-center")
            article_text = []
            for obj in objects:
                  paragraph_text = obj.find_all('p')
                  for p in paragraph_text:
                        article_text.append(p.text)
                  #print(obj.text)
            article_text = ' '.join(article_text)
   
            for word in search_terms['search_term']:
                  if word in article_text:
                        save = list(row)
                        save.append(article_text)
                        relevant_text.append(save)                  
                        print('Search term found: ' +  word)
                        break
            else: 
                  pass

            time.sleep(5)
      else:
            pass

In [ ]:
len(relevant_text)

# CNA

In [4]:
from  urllib.request import urlopen
import shutil

In [5]:
#base_url_1 = 'https://www.cna.org/centers/cna/sppp/rsp/russia-ai-archive#newsletters'
base_url_2  = "https://www.cna.org/our-media/newsletters/china-ai-and-autonomy-report?index=true"

In [6]:
# convert PDF 
# clean up PDF

In [7]:
today = datetime.now()

today

datetime.datetime(2022, 5, 6, 11, 42, 35, 686736)

In [8]:
back_to = today.date() - timedelta(days=30)
back_to

datetime.date(2022, 4, 6)

In [9]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


In [10]:
# create loop over base_url_1 and_2

In [11]:
ua = UserAgent(verify_ssl=False, cache=False)

user_agent = ua.random
header = {'User-Agent': user_agent}

response = requests.get(base_url_2, headers=header)

html = soup(response.text, 'lxml')

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/home/jesselehrke/Documents/GitHub/disinfo_radar_development/dri_venv/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [12]:
title_list = []
url_list = []
date_list = []

In [ ]:

objects = html.find_all('div', class_='two-column-layout')#, href=True)#elid=True)#class_= 'clearfix')
print(len(objects))

for obj in objects:
      items = obj.find_all('li')#, href=True)
      print(len(items))
      for i in items:
            print(i.text)
            title_list.append(i.text)

            urls = i.find('a', href=True)
            #print(urls['href'])
            url_list.append('https://www.cna.org' + urls['href'])

            date = i.text
            date = date.split(', ')[1:]
            date = ' '.join(date).strip()
            #date = date.strip()
            print(date)
            date_parsed = datetime.strptime(date, '%B %d %Y')
            date_list.append(date_parsed)

In [17]:
# Save collection to dataframe

df_collected = pd.DataFrame(list(zip(title_list, url_list, date_list)), 
            columns=['title', 'url', 'date'])

df_collected.head(10)

,title,url,date


In [105]:
# WEB REDESIGN
test_url = 'https://www.cna.org/our-media/newsletters/china-ai-and-autonomy-report/issue-14'

In [106]:
response = requests.get(test_url, headers=header)

html = soup(response.text, 'lxml')

In [107]:
main_header = html.find('h1').text
issue_header = html.find('h2').text.strip()
issue_header = issue_header.split('\r')[0]
title = main_header + ': ' + issue_header

In [108]:

objects = html.find('section', class_='content-rows')

header_list = []
try:
      full_text = objects.text.split('NOTES')[0]
except:
      full_text = objects.text

headers = objects.find_all('strong')
for head in headers:
      if head.text == '':
            pass
      else:
            header_list.append(head.text)


In [109]:
def split_article_on(txt, seps):
    '''
    Splits a full text article on a list of seperators
    '''
    default_sep = seps[0]

    # we skip seps[0] because that's the default separator
    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)
    return [i.strip() for i in txt.split(default_sep)]


In [110]:
articles = split(full_text, header_list)

In [111]:
while len(header_list) != len(articles):
      header_list.insert(0, 'Intro (no tag)')

In [112]:
print(len(header_list))
print(len(articles))

18
18


In [113]:
header_list = [title + ' - ' + head for head in header_list]

In [114]:
header_list

['The China AI and Autonomy Report: Issue 14, May 5, 2022 - Intro (no tag)',
 'The China AI and Autonomy Report: Issue 14, May 5, 2022 - DJI suspends operations in Russia and Ukraine amidst continued controversy over the use of its UAVs in the Ukraine conflict.',
 'The China AI and Autonomy Report: Issue 14, May 5, 2022 -  PLA National Defense University (NDU) professor argues that cognitive warfare operations in the Russia-Ukraine War represents the future of warfare. ',
 'The China AI and Autonomy Report: Issue 14, May 5, 2022 -   Trend 1: From the "strong beating the weak" to the "intelligent beating the dull."  ',
 'The China AI and Autonomy Report: Issue 14, May 5, 2022 -  Trend 2: From "destructive power" to "manipulating cognition. ',
 'The China AI and Autonomy Report: Issue 14, May 5, 2022 -  Trend 3: From "human-based" to "human-machine collaboration." ',
 'The China AI and Autonomy Report: Issue 14, May 5, 2022 -  Trend 4: From "big eats small" to "fast eats slow." ',
 'The 

In [115]:
##########

In [38]:
objects = html.find_all('section', class_='content-rows')

for o in objects:
      paragraph = o.text
      headers = o.find_all('strong')
      for head in headers:
            print(head)








Welcome to the China AI and Autonomy Report, a biweekly newsletter published by CNA.
Authors in the PLA Daily consider the best way forward for evaluating intelligent command systems, emphasizing the importance of integrating actual command parameters into assessing their effectiveness. PLA researchers have made a breakthrough in using an AI system with a satellite video to better identify and track objects such as planes and cars. The Cyberspace Administration of China has launched a campaign targeting the use of recommendation algorithms in internet companies. The campaign has an explicit focus on large, socially influential internet platforms, which has caused speculation that it is likely an effort to rein in social media giants Tencent and TikTok's parent company Bytedance. PRC ministries have also released a scientific and technology development plan for its transportation sector that seeks to integrate "intelligent" technologies into many aspects of transportation, including

In [20]:
objects = html.find('div', class_='container')
#para = objects.findNext('p') #!!works
#print(para.text)

items = objects.find_all('p')#, href=True)
print(len(items))
for i in items:
      print(i.text)
      parent = i.parent
      bullets = parent.findNext('ul')
      #bullets = i.findNext('ul')#, href=True)
      print(bullets.text)

#items = objects.find_all('ul')#, href=True)


30
Welcome to the China AI and Autonomy Report, a biweekly newsletter published by CNA.

Evaluations of situational awareness capabilities.  Criteria for evaluating situational awareness capabilities include the ability to assess enemy and friendly dispositions, perceive changes in the situation in real-time, and make predictions about the future battlefield situation.
Evaluations of operational decision-making.  Criteria for evaluating operational decisions include the ability to make systematic, rational, and innovative decisions. Effective evaluations include assessing the ability of intelligent command systems to deal with major changes that occur during operations, use standardized procedures, inherit content, and make timely decisions.
Evaluations of the ability to formulate plans. Criteria for evaluating the ability to formulate plans include the ability to retrieve and prioritize pre-determined solutions, accurately and rationally match plans to the situation and operational ob

In [ ]:
objects = html.find_all('div', class_='container')
print(len(objects))

for obj in objects:
      items = obj.find_all('p')#, href=True)
      print(len(items))
      for i in items:
            print(i.text)

### Get PDFs

In [ ]:
QUERY = 'CNA'
pdf_dir = DATA_PATH + QUERY + 'pdfs'

if not os.path.exists(pdf_dir): 
  os.makedirs(pdf_dir)

have = set(os.listdir(pdf_dir))

# Time out for requests
timeout_secs = 10 


In [ ]:
for index, row in df_collected.iterrows():
      #if row.date >= today - timedelta(days=14):

      basename = row.url.split('/')[-1]
      fname = os.path.join(pdf_dir, basename)
      print(fname)

      # try:
      if not basename in have:
            print('fetching %s into %s' % (row.url, fname))
            req = urlopen(row.url, None, timeout_secs)
            with open(fname, 'wb') as fp:
                  shutil.copyfileobj(req, fp)
      else:
            print('%s exists, skipping' % (fname, ))


### PDF to text

In [ ]:
def bulk_text_to_pdf(dir):
    done = []
    #done_path = []
    problem = []

    for root, dirs, files in os.walk(dir):
        #vari.set("Writing contents")
        for file in files:
            path_to_pdf = os.path.join(root, file)
            [stem, ext] = os.path.splitext(path_to_pdf)
            if ext == '.pdf':
                #vari.set("Processing " + path_to_pdf)
                print("Processing " + path_to_pdf)
                pdf_contents = parser.from_file(path_to_pdf)
                print(pdf_contents)
                path_to_txt = stem + '.txt'
                name = stem.split('\\')[-1]
                with open(path_to_txt, 'w', encoding='utf-8') as txt_file:
                    print("Writing contents to " + path_to_txt)
                    #vari.set("Writing contents to " + path_to_txt)
                    if pdf_contents['content'] is None:
                        pass
                    else:
                        #pdf_contents = pdf_contents['content'].replace('\n\n\n\n', '\n')
                        #pdf_contents = pdf_contents['content'].replace('\d*\s\s\w*\.', '', regex=True)
                        txt_file.write(pdf_contents['content'])
                size = os.path.getsize(path_to_txt)
                if size <= 200:
                    problem.append(name)
                    problem_path.append(path_to_pdf)                    
                else:
                    done.append(name)
                    #done.append(path_to_pdf)                    
    #vari.set('Done')
    str1 = str('Successfully converted: ' + ', '.join(done))
    str2 = str('Possible problems with: ' + ', '.join(problem))
    #messagebox.showinfo('Done', str1 + '\n' + str2)
    print(problem_path)

In [ ]:
from tika import parser


In [ ]:
bulk_text_to_pdf(pdf_dir)

In [ ]:
# Checking for a program and a folder
if not shutil.which('pdftotext'): # needs Python 3.3+
  print('ERROR: you don\'t have pdftotext installed. Install it first before calling this script')
  sys.exit()

if not os.path.exists(DATA_PATH + QUERY + '_txt'):
      os.makedirs(DATA_PATH + QUERY + '_txt')

In [ ]:
# Specifying paths
txt_dir = DATA_PATH + QUERY + '_txt'
pdf_dir = DATA_PATH + QUERY + 'pdfs'

In [ ]:
# Note- should make a function in utils and import

have = set(os.listdir(txt_dir))
files = os.listdir(pdf_dir)

for i,f in enumerate(files):

  txt_basename = f + '.txt'
  
  if txt_basename in have:
    print('%d/%d skipping %s, already exists.' % (i, len(files), txt_basename, ))
    continue

  pdf_path = os.path.join(pdf_dir, f)
  txt_path = os.path.join(txt_dir, txt_basename)
  
  cmd = "pdftotext %s %s" % (pdf_path, txt_path)
  os.system(cmd)

  print('%d/%d %s' % (i, len(files), cmd))

  # check output was made
  if not os.path.isfile(txt_path):
    # there was an error with converting the pdf
    print('there was a problem with parsing %s to text, creating an empty text file.' % (pdf_path, ))
    os.system('touch ' + txt_path) # create empty file, but it's a record of having tried to convert

  time.sleep(0.01) #  for ctrl+c termination

In [ ]:
# To be nice I should parse these a bit and clean them up
# THEN open txt files and insert into DF (naturally create DF)

# See txts_to_csv notebook, easy task

# Scrap

In [ ]:
def get_response():
    #base = test_link
    final_url = base_url
    #driver = webdriver.Chrome()
    
    with requests.get(final_url) as response:
        #response = requests.get(url, headers=headers)
        page_soup = soup(response.content, 'lxml')
        return page_soup

In [ ]:
from seleniumwire import webdriver  # Import from seleniumwire

# Create a new instance of the Firefox driver
#driver = webdriver.Chrome()

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166"')
driver = webdriver.Chrome(chrome_options=chrome_options)
user_agent = driver.execute_script("return navigator.userAgent;")
# Go to the Google home page

driver.get(base_url_1)

# with driver.requests.get(base_url) as response:
#     #response = requests.get(url, headers=headers)
#     page_soup = soup(response.content, 'lxml')
#     #return page_soup

# Access requests via the `requests` attribute
for request in driver.requests:
    if request.response:
        print(
            request.path,
            request.response.status_code,
            request.response.headers['Content-Type'])


In [ ]:
driver.close()
